In [3]:
import os
import tabula
import pandas as pd
from tqdm import tqdm
import PyPDF2
import pytesseract
from pdf2image import convert_from_path
from PIL import Image

# Create subfolders for output
output_folder_name = "NEET_2024_Excel"
if not os.path.exists(output_folder_name):
    os.makedirs(output_folder_name)

pdf_folder_path = "NEET_2024_PDFs"
individual_folder_name = os.path.join(output_folder_name, "Individual_Excel")
if not os.path.exists(individual_folder_name):
    os.makedirs(individual_folder_name)

# Initialize an empty DataFrame for the master file
master_df = pd.DataFrame()

# Function to extract text from the first page of a PDF
def extract_text_from_first_page(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfFileReader(file)
        first_page = reader.getPage(0)
        text = first_page.extract_text()
    return text

# Function to extract center details from the text
def extract_center_details(text):
    lines = text.split('\n')
    center_no = None
    center_name = None
    for line in lines:
        if "Centre:" in line:
            center_no = line.split("Centre:")[1].split()[0].strip()
            center_name = line.split("Centre:")[1].split(" ", 1)[1].strip()
            break
    return center_no, center_name

# Function to extract tables using OCR if tabula fails
def extract_text_using_ocr(pdf_path, page_num):
    images = convert_from_path(pdf_path, first_page=page_num, last_page=page_num)
    text = pytesseract.image_to_string(images[0])
    return text

# Function to get the number of pages in the PDF
def get_num_pages(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfFileReader(file)
        return reader.numPages

# Function to process each PDF
def process_pdf(pdf_path):
    num_pages = get_num_pages(pdf_path)
    # Extract tables from all pages of the PDF
    tables = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True)
    
    if tables:
        # Combine all tables into a single DataFrame
        combined_tables = pd.DataFrame()
        for table in tables:
            if not table.empty:
                combined_tables = pd.concat([combined_tables, table], ignore_index=True)
        
        # Check if last page has no data and attempt to use OCR
        if num_pages >= 3:
            last_page_table = tabula.read_pdf(pdf_path, pages=str(num_pages), multiple_tables=True)
            if not last_page_table or all(table.empty for table in last_page_table):
                last_page_text = extract_text_using_ocr(pdf_path, num_pages)
                # Process the OCR extracted text
                ocr_lines = last_page_text.split('\n')
                ocr_data = []
                for line in ocr_lines:
                    if line.strip():
                        parts = line.split()
                        if len(parts) == 2 and parts[0].isdigit() and parts[1].isdigit():
                            ocr_data.append(parts)
                if ocr_data:
                    ocr_df = pd.DataFrame(ocr_data, columns=['Srlno.', 'Marks'])
                    combined_tables = pd.concat([combined_tables, ocr_df], ignore_index=True)
        
        # Extract center details from the PDF text
        text = extract_text_from_first_page(pdf_path)
        center_no, center_name = extract_center_details(text)
        
        if not center_no:
            center_no = os.path.splitext(os.path.basename(pdf_path))[0]
        
        city_code = center_no[:4] if center_no else "Unknown"
        
        # Consolidate Srlno. and Marks columns
        srlno_cols = [col for col in combined_tables.columns if 'Srlno.' in col or 'Sr. No' in col]
        marks_cols = [col for col in combined_tables.columns if 'Marks' in col]
        
        consolidated_df = pd.DataFrame()
        for srlno_col, marks_col in zip(srlno_cols, marks_cols):
            temp_df = combined_tables[[srlno_col, marks_col]].copy()
            temp_df.columns = ['Srlno.', 'Marks']
            consolidated_df = pd.concat([consolidated_df, temp_df], ignore_index=True)
        
        # Convert 'Srlno.' column to numeric, coercing errors to NaN
        consolidated_df['Srlno.'] = pd.to_numeric(consolidated_df['Srlno.'], errors='coerce')
        
        # Remove rows where Srlno. is NaN
        consolidated_df = consolidated_df.dropna(subset=['Srlno.'])
        
        # Sort by Srlno.
        consolidated_df = consolidated_df.sort_values(by='Srlno.')
        
        consolidated_df['Center No'] = center_no
        consolidated_df['Center Name'] = center_name
        consolidated_df['City Code'] = city_code
        
        return consolidated_df
    return pd.DataFrame()


In [7]:
# Initialize an empty DataFrame for the master file
master_df = pd.DataFrame()

# Process each PDF in the folder
for pdf_file in tqdm(os.listdir(pdf_folder_path), desc="Processing PDFs", unit="file"):
    if pdf_file.endswith(".pdf"):
        pdf_path = os.path.join(pdf_folder_path, pdf_file)
        
        # Check if the Excel file already exists
        individual_excel_path = os.path.join(individual_folder_name, f"{os.path.splitext(pdf_file)[0]}.xlsx")
        if os.path.exists(individual_excel_path):
            #print(f"File {individual_excel_path} already exists. Skipping...")
            continue
        
        df = process_pdf(pdf_path)
        
        if not df.empty:
            # Save each individual DataFrame to an Excel file
            df.to_excel(individual_excel_path, index=False)
            
            # Append to the master DataFrame
            master_df = pd.concat([master_df, df], ignore_index=True)

# Save the master DataFrame to an Excel file
master_excel_path = os.path.join(output_folder_name, "NEET_2024_Master_Results.xlsx")
master_df.to_excel(master_excel_path, index=False)

print(f"Individual Excel files have been saved to {individual_folder_name}")
print(f"Master Excel file has been saved to {master_excel_path}")

Processing PDFs: 100%|██████████| 4700/4700 [07:24<00:00, 10.57file/s]


Individual Excel files have been saved to NEET_2024_Excel/Individual_Excel
Master Excel file has been saved to NEET_2024_Excel/NEET_2024_Master_Results.xlsx


In [8]:
master_df 

,Srlno.,Marks,Center No,Center Name,City Code
0,1.0,672.0,410201,"410201 Page No. 1 SSVM WORLD SCHOOL, 72/2 VA...",4102
1,2.0,562.0,410201,"410201 Page No. 1 SSVM WORLD SCHOOL, 72/2 VA...",4102
2,3.0,519.0,410201,"410201 Page No. 1 SSVM WORLD SCHOOL, 72/2 VA...",4102
3,4.0,76.0,410201,"410201 Page No. 1 SSVM WORLD SCHOOL, 72/2 VA...",4102
4,5.0,413.0,410201,"410201 Page No. 1 SSVM WORLD SCHOOL, 72/2 VA...",4102
...,...,...,...,...,...
658587,480.0,102.0,411106,411106 Page No. 1 THE PUPIL SAVEETHA ECO SCH...,4111
658588,481.0,115.0,411106,411106 Page No. 1 THE PUPIL SAVEETHA ECO SCH...,4111
658589,482.0,76.0,411106,411106 Page No. 1 THE PUPIL SAVEETHA ECO SCH...,4111
658590,483.0,77.0,411106,411106 Page No. 1 THE PUPIL SAVEETHA ECO SCH...,4111


In [37]:
# Specific PDF file for testing
pdf_file = "200111.pdf"
pdf_path = os.path.join(pdf_folder_path, pdf_file)

# Process the specific PDF
df = process_pdf(pdf_path)

if not df.empty:
    # Save the individual DataFrame to an Excel file
    individual_excel_path = os.path.join(individual_folder_name, f"{os.path.splitext(pdf_file)[0]}.xlsx")
    df.to_excel(individual_excel_path, index=False)
    
    print(f"Processed and saved {pdf_file} to {individual_excel_path}")
else:
    print(f"No data extracted from {pdf_file}")


Processed and saved 200111.pdf to NEET_2024_Excel/Individual_Excel/200111.xlsx
